In [ ]:
from assets.view import activate

activate()

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/srchd/machine-learning/blob/main/plane/visu.ipynb)